In [163]:
import json
import pandas as pd
import os, re, glob
from collections import defaultdict
def read_exploit_module(exploit_dir='data/exploit/modules/exploits/**/*.rb'):
    #nested=defaultdict(list)
    #data = defaultdict(nested)
    rec_dd = lambda: defaultdict(rec_dd)
    data = rec_dd()
    #for filename in os.listdir(exploit_dir):
    for filename in glob.iglob(exploit_dir, recursive=True):
        with open(filename) as f:
            for line in f.readlines():
                CVE=""
                PRODUCT=""
                VENDOR=""
                RANK=""
                #  Rank = ExcellentRanking
                if "Rank" in line:
                    RANK=re.search(r'Rank[\s]*=[\s]*([\w]*)', line)
                    if RANK:
                        RANK=RANK.groups()[0]
                if "CVE" in line:
                    CVE=re.search(r'\'([\d]*-[\d]*)\'', line)
                    if CVE:
                        CVE="CVE-"+CVE.groups()[0]
                        EXPLOIT_PATH=filename.split('.')[0]
                        EXPLOIT_NAME=EXPLOIT_PATH.split('/')[-1]
                        EXPLOIT_PRODUCT=EXPLOIT_PATH.split('/')[-2]
                        EXPLOIT_PLATFORM=EXPLOIT_PATH.split('/')[-3]
                        #data[CVE]=({EXPLOIT_PLATFORM:EXPLOIT_NAME})
                        #data[CVE][EXPLOIT_NAME]={"RANK":RANK,"PLATFORM":EXPLOIT_PLATFORM,
                        #                         "PRODUCT":EXPLOIT_PRODUCT}
                        data[CVE][EXPLOIT_PRODUCT]=EXPLOIT_NAME
                        #data.update(CVE:EXPLOIT})
                        pass
    return data
                        

def parse_cve_json(cve_data,columns=['CVE','vendor','product','version'], save_loc="data/vuln/data.pkl"):
    df = pd.DataFrame(columns=['CVE','vendor','product','version'])
    for item in cve_data["CVE_Items"]:
        CVE = item["cve"]['CVE_data_meta']['ID']
        try:
            for vendor in item['cve']['affects']['vendor']['vendor_data']:
                vendor_name = vendor['vendor_name']
                for product in vendor['product']['product_data']:
                    product_name = product['product_name']
                    versions = [x['version_value'] for x in product['version']['version_data']]

                    new_df = pd.DataFrame([[CVE,vendor_name,product_name,versions]],
                                              columns=['CVE','vendor','product','version'])
                    df = df.append(new_df, ignore_index=True)           
        except IndexError:
            pass
    if save_loc:
        save_vuln_data(save_loc)
    return df


def save_vuln_data(df = None, save_loc = "data/exploit/data.pkl"):
    if not df:
        cve_data = read_vuln_json()
        df = parse_cve_json(cve_data)
    return df.to_pickle("data/exploit/data.pkl")


def load_expl_data():
    try:
        return pd.read_pickle("data/exploit/data.pkl")
    except:
        save_vuln_data() #force a load/parse/save
        return pd.read_pickle("data/exploit/data.pkl")
        

In [164]:
json_vulns = json.loads(json.dumps(read_exploit_module())) #terrible hack
with open('data/exploit/data.json', 'w') as outfile:
    json.dump(json_vulns, outfile)

In [165]:
json_vulns

{'CVE-1978-1234': {'exploits': 'example'},
 'CVE-2001-0797': {'login': 'manyargs', 'telnet': 'ttyprompt'},
 'CVE-2007-2447': {'samba': 'usermap_script'},
 'CVE-2002-1318': {'samba': 'nttrans'},
 'CVE-2002-1643': {'realserver': 'describe'},
 'CVE-2014-6271': {'ftp': 'pureftpd_bash_env_exec',
  'http': 'ipfire_bashbug_exec',
  'local': 'vmware_bash_function_root',
  'smtp': 'qmail_bash_env_exec',
  'dhcp': 'bash_environment'},
 'CVE-2000-0573': {'ftp': 'wuftpd_site_exec_format'},
 'CVE-2001-0414': {'ntp': 'ntp_overflow'},
 'CVE-2004-0397': {'svn': 'svnserve_date'},
 'CVE-2007-1286': {'php': 'php_unserialize_zval_cookie'},
 'CVE-2012-5958': {'upnp': 'libupnp_ssdp_overflow'},
 'CVE-2006-5276': {'ids': 'snort_dce_rpc'},
 'CVE-2009-0695': {'wyse': 'hagent_untrusted_hsdata'},
 'CVE-2013-0431': {'browser': 'java_jre17_jmxbean_2'},
 'CVE-2010-0840': {'browser': 'java_trusted_chain'},
 'CVE-2006-3677': {'browser': 'mozilla_navigatorjava'},
 'CVE-2014-8636': {'browser': 'firefox_proxy_prototype'}